In [105]:
import numpy as np
import pandas as pd

In [106]:
# data cleaning
data = pd.read_csv("./return_data.csv")
data["index"] = pd.to_datetime(data["index"], format="%Y-%m-%d")
data = data.set_index("index")

In [107]:
# extract correlation matrix
sigma = data.cov()
cor = data.corr()
std = np.zeros((data.shape[1],data.shape[1]))
np.fill_diagonal(std, data.std())

# check
print(np.any((sigma.values - (std.T @ cor @ std).values) > 1e-6))

False


In [108]:
# eigen decomposition
eg_val, eg_vec = np.linalg.eig(cor)
value = np.zeros((data.shape[1],data.shape[1]))
np.fill_diagonal(value, np.sqrt(eg_val))
result = eg_vec @ np.multiply(value, value) @ eg_vec.T

# check
print(np.any((result - cor.values) > 1e-6))

c = eg_vec @ value @ eg_vec.T

False


In [109]:
# recursive algorithm
def mt(c, threshold, loop_num:int):
    d = np.eye(c.shape[1])
    pie = 0
    for i in range(loop_num):
        u = (d @ (c*c) @ d)**0.5
        inv_u = np.linalg.inv(u)
        if np.isnan(inv_u).sum() > 0:
            break
        q = inv_u.T @ d @ c
        qc = np.zeros((c.shape[0], c.shape[1]))
        np.fill_diagonal(qc, np.diag(q.T @ c))
        d = qc
        if np.all((d @ q - pie) < threshold):
            break
        else:
            pie = d @ q
    return pie

In [110]:
# check
C_test = np.array([[1, 0.5, 0.3], [0.5, 1, 0.1], [0.3, 0.1, 1]])
eg_val_test, eg_vec_test = np.linalg.eig(C_test)
value_test = np.zeros((C_test.shape[1],C_test.shape[1]))
np.fill_diagonal(value_test, np.sqrt(eg_val_test))

c_test = eg_vec_test @ value_test @ eg_vec_test.T

result_test = mt(c_test, 1e-6, int(1e9))
expected_test = np.array([
        [0.9535, -0.0016, -0.0026],
        [0.0017, 0.9661, -0.0001],
        [0.0027, 0.0001, 0.9886]])

np.linalg.norm(result_test - expected_test, 'fro')

0.0007428807319840459

In [116]:
pie_data = mt(c, 1e-6, int(1e9))
t_data = np.diag(std) @ pie_data @ np.linalg.inv(c) @ (np.diag(std))**-1
t_data

16.025571266200274